In [ ]:
from itertools import product
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
from tensorflow.keras import backend as K

In [ ]:
# read in the prepared data
df = pd.read_csv("data_prep_MA3H-DIFFSIGN-SUM135.csv", index_col="timestamp")

df_realitytest = df.iloc[-2688:]
X_realitytest = df_realitytest.drop(columns=["consumption"])
y_realitytest = df_realitytest["consumption"]

df_training = df.iloc[:-2688]
X = df_training.drop(columns=["consumption"])
y = df_training["consumption"]
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a scaler object
scaler = MinMaxScaler()

# Normalize the data
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)
X_realitytest_norm = scaler.transform(X_realitytest)

# # Back-scale the data
# X_train_back = scaler.inverse_transform(X_train_norm)
# X_test_back = scaler.inverse_transform(X_val_norm)

In [ ]:
# Define the hyperparameters to tune
params = {
    'hidden_layers': [1, 2, 3],
    'neurons_per_layer': [50, 25, 100],
    # 'learning_rate': [0.001, 0.01, 0.1],
    'activation': ['tanh', 'relu', 'sigmoid', 'linear', 'softmax', 'selu', 'gelu', 'exponential'],
    'batch_size': [16, 32, 64]
}

# Define the search space
param_combinations = product(*params.values())

In [ ]:
# Define the evaluation metric
eval_metric = mean_squared_error

In [ ]:
# Define the model architecture
def create_model(hidden_layers, neurons_per_layer, activation):
    model = Sequential()
    model.add(Input(shape=X_train.shape[1:]))
    for i in range(hidden_layers):
        model.add(Dense(neurons_per_layer, activation=activation))
    model.add(Dense(1, activation='linear'))
    model.compile(loss=rmse, optimizer=Adam(), metrics=[rmse])
    return model

In [ ]:
# Define the training procedure
num_epochs = 30

In [ ]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [16]:
# test train
hidden_layers = 1
neurons_per_layer = 200
learning_rate = 0.01
activation = 'tanh'
batch_size = 32

model= Sequential([
    Input(shape = X_train.shape[1:]),
    Dense(50, activation='tanh'),
    Dense(100, activation='selu'),
    Dense(25, activation='tanh'),
    Dense(100, activation='selu'),
    Dense(25, activation='exponential'),
    Dense(1)
])

model.compile(
    loss=rmse,
    optimizer=Adam(),
    metrics=[rmse]
)

# model = create_model(hidden_layers, neurons_per_layer, learning_rate, activation)

history = model.fit(X_train_norm, y_train, validation_data=(X_test_norm, y_test), batch_size=batch_size, epochs=num_epochs)

# Calculate the validation loss
val_loss = eval_metric(y_test, model.predict(X_test_norm))

# Generate a plot of the predictions vs. actual values
y_pred = model.predict(X_realitytest_norm)
# y_pred = scaler.inverse_transform(y_pred_norm)
# y_test_orig = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

fig = go.Figure()
fig.add_trace(go.Scatter(x=X_realitytest.index, y=y_realitytest, name='Actual', mode='lines'))
fig.add_trace(go.Scatter(x=X_realitytest.index, y=y_pred.flatten(), name='Predicted (neurons per layer: {})'.format(neurons_per_layer), mode='lines'))
fig.update_layout(title='Energy Consumption Prediction (Activation: {}, Hidden Layers: {})'.format(activation, hidden_layers), xaxis_title='Time', yaxis_title='Energy Consumption')
fig.show()

Epoch 1/30
1415/1415 [==============================] - 3s 2ms/step - loss: 31.8557 - rmse: 31.8525 - val_loss: 26.4570 - val_rmse: 26.4608
Epoch 2/30
1415/1415 [==============================] - 2s 2ms/step - loss: 23.8824 - rmse: 23.8822 - val_loss: 23.4190 - val_rmse: 23.4291
Epoch 3/30
1415/1415 [==============================] - 2s 2ms/step - loss: 22.2250 - rmse: 22.2237 - val_loss: 20.6613 - val_rmse: 20.6726
Epoch 4/30
1415/1415 [==============================] - 2s 2ms/step - loss: 21.8526 - rmse: 21.8525 - val_loss: 22.7147 - val_rmse: 22.7312
Epoch 5/30
1415/1415 [==============================] - 2s 2ms/step - loss: 21.1400 - rmse: 21.1406 - val_loss: 22.6032 - val_rmse: 22.6197
Epoch 6/30
1415/1415 [==============================] - 2s 2ms/step - loss: 21.4664 - rmse: 21.4659 - val_loss: 20.2689 - val_rmse: 20.2763
Epoch 7/30
1415/1415 [==============================] - 2s 2ms/step - loss: 21.0093 - rmse: 21.0092 - val_loss: 20.0181 - val_rmse: 20.0290
Epoch 8/30
1415/1415

In [ ]:
# test_y_pred = y_realitytest.shift(1)
#
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=X_realitytest.index, y=test_y_pred, name='test_y_pred', mode='lines'))
# fig.add_trace(go.Scatter(x=X_realitytest.index, y=y_realitytest, name='Actual', mode='lines'))
# fig.add_trace(go.Scatter(x=X_realitytest.index, y=y_pred.flatten(), name='Predicted (neurons per layer: {})'.format(neurons_per_layer), mode='lines'))
# fig.update_layout(title='Energy Consumption Prediction (Activation: {}, Hidden Layers: {})'.format(activation, hidden_layers), xaxis_title='Time', yaxis_title='Energy Consumption')
# fig.show()
# y_pred

In [ ]:
# Implement the grid search
results = []

for params in param_combinations:
    hidden_layers, neurons_per_layer, activation, batch_size = params
    model = create_model(hidden_layers, neurons_per_layer, activation)
    try:
        history = model.fit(X_train_norm, y_train, validation_data=(X_test_norm, y_test), batch_size=batch_size, epochs=num_epochs)
        # Calculate the validation loss
        val_loss = rmse(y_realitytest, model.predict(X_realitytest_norm).flatten())

        results.append([hidden_layers, neurons_per_layer, activation, batch_size, val_loss.numpy()])

        # Generate a plot of the predictions vs. actual values
        y_pred = model.predict(X_realitytest_norm)
        # y_pred = scaler.inverse_transform(y_pred_norm)
        # y_test_orig = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

        model.save("model_{}_{}_{}_{}".format(hidden_layers, neurons_per_layer, activation, batch_size))

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=X_realitytest.index, y=y_realitytest, name='Actual', mode='lines'))
        fig.add_trace(go.Scatter(x=X_realitytest.index, y=y_pred.flatten(), name='Predicted (neurons per layer: {})'.format(neurons_per_layer), mode='lines'))
        fig.update_layout(title='Energy Consumption Prediction (Activation: {}, Hidden Layers: {})'.format(activation, hidden_layers), xaxis_title='Time', yaxis_title='Energy Consumption')
        fig.show()

        df_temp = pd.DataFrame([[X_realitytest.index, y_realitytest, y_pred.flatten()]], columns=["timestamp", "y_realitytest", "y_pred"])
        df_temp.to_csv("realitytest_{}_{}_{}_{}.csv".format(hidden_layers, neurons_per_layer, activation, batch_size), index=False)

        results_df = pd.DataFrame(results, columns=["hidden_layers", "neurons_per_layer", "activation", "batch_size", "val_loss"])
        results_df.to_csv("results_df.csv", index=False)
    except:
        continue


data_prep_MA3H-MINMAX5n1-SUM35
data_prep_MA3H-DIFFSIGN-SUM135
data_prep_MA3H-DIFFSIGN-MINMAX
data_prep_DIFF1HOUR_MA1H